In [1]:
import pathlib
import re

import pandas as pd

instance_dir = pathlib.Path()/"instances"
result_dir = pathlib.Path()/"ampl_outputs"
result_file_paths = list(result_dir.glob("*"))

In [44]:
result_dict_list = []
for result_file in result_file_paths:
    filename = result_file.name
    if "m0" in filename:
        continue
    instance_name = filename[:-13]
    # print(instance_name)
    instance_file = instance_dir/f"{instance_name}ampl_.txt"
    with open(result_file.absolute(), "r") as rf, open(instance_file.absolute(), "r") as ins_f:
        result_text = rf.read()
        instance_text = ins_f.read()
        # print(instance_text)
        # break
        mrt_stations = re.findall(r"set M:=[^;]*", instance_text)[0]
        mrt_stations = re.findall(r"\d+", mrt_stations)
        mrt_stations = [int(mrt_station) for mrt_station in mrt_stations]
        dummy_mrt_stations = re.findall(r"set M_B:=[^;]*", instance_text)[0]
        dummy_mrt_stations = re.findall(r"\d+", dummy_mrt_stations)
        dummy_mrt_stations = [int(mrt_station) for mrt_station in dummy_mrt_stations]
        # print(mrt_stations)
        # print(dummy_mrt_stations)

        match = re.search(r"param d:=(.*?);", instance_text, re.S)
        demands = None
        if match:
            block = match.group(1)
            pairs = re.findall(r"(\d+)\s+([\d.]+)", block)

            # convert to dict
            d_dict = {int(i): float(v) for i, v in pairs}

            # build list with 0 at index 0
            max_index = max(d_dict.keys())
            demands = [0] * (max_index + 1)
            for i, v in d_dict.items():
                demands[i] = int(v)

        # print(demands)

        # Capture the block after the header until the next blank line or non-number line
        match = re.search(r"Customer assigment to PL: u variables:\s*((?:\d+\s+\d+\s*\n?)+)", result_text)
        cust_locker_assignment_pairs = None
        if match:
            block = match.group(1)
            cust_locker_assignment_pairs = re.findall(r"(\d+)\s+(\d+)", block)
            cust_locker_assignment_pairs = [(int(i), int(j)) for i, j in cust_locker_assignment_pairs]
        # print(cust_locker_assignment_pairs)

        match = re.search(r"param G:=(.*?);", instance_text, re.S)
        locker_capacites = {}
        if match:
            block = match.group(1)
            pairs = re.findall(r"(\d+)\s+(\d+)", block)
            pairs = [(int(i), int(j)) for i, j in pairs]
            for pair in pairs:
                locker_capacites[pair[0]] = pair[1]
        # print(locker_capacites)
        locker_loads = [0]*200
        for pair in cust_locker_assignment_pairs:
            locker_loads[pair[1]] += demands[pair[0]]
        # print(locker_loads)
        pairs = re.findall(r"MRT used:\s*(\d+)\s*→\s*(\d+)", result_text)
        mrt_lines = [(int(a), int(b)) for a, b in pairs]
        for mrt_line in mrt_lines:
            m = mrt_line[0]
            li = 0
            for li in range(len(mrt_stations)):
                if mrt_stations[li]==m:
                    break
            mb = 9999999999
            if (li%2==1):
                mb = dummy_mrt_stations[li-1]
            else:
                mb = dummy_mrt_stations[li+1]
            act1 = locker_loads[m] + locker_loads[mb]
            if act1 > locker_capacites[mb]:
                print(instance_name, " (", m,",",mb, ") ", act1, ">", locker_capacites[mb])

        
        
        
        

A-n24-k4-m2-b4  ( 27 , 32 )  91 > 78
A-n32-k6-m2-b6  ( 34 , 37 )  137 > 99
A-n33-k5-m1-b5  ( 35 , 36 )  101 > 96
A-n33-k5-m2-b5  ( 34 , 39 )  124 > 83
A-n33-k5-m3-b5  ( 35 , 40 )  139 > 96
A-n36-k5-m1-b5  ( 37 , 40 )  73 > 70
A-n36-k6-m1-b6  ( 37 , 40 )  135 > 97
A-n38-k6-m1-b6  ( 40 , 41 )  79 > 76
A-n45-k7-m1-b7  ( 46 , 49 )  121 > 92
A-n47-k7-m1-b7  ( 48 , 51 )  144 > 81
A-n47-k7-m2-b7  ( 48 , 53 )  98 > 81
A-n52-k7-m2-b7  ( 54 , 57 )  122 > 97
A-n54-k9-m1-b9  ( 55 , 58 )  102 > 99
A-n61-k8-m1-b8  ( 63 , 64 )  122 > 98
A-n61-k8-m2-b8  ( 63 , 66 )  113 > 98
A-n61-k8-m3-b8  ( 62 , 69 )  88 > 86
A-n61-k8-m3-b8  ( 67 , 72 )  81 > 71
A-n62-k10-m1-b10  ( 63 , 66 )  86 > 73
A-n62-k10-m2-b10  ( 63 , 68 )  124 > 73
A-n62-k10-m3-b10  ( 63 , 70 )  91 > 73
A-n62-k9-m2-b9  ( 66 , 69 )  93 > 70
A-n62-k9-m3-b9  ( 68 , 73 )  91 > 71
A-n63-k9-m1-b9  ( 64 , 67 )  107 > 73
A-n64-k9-m2-b9  ( 65 , 70 )  114 > 89
A-n68-k9-m1-b9  ( 70 , 71 )  110 > 84
A-n68-k9-m2-b9  ( 70 , 73 )  111 > 84
A-n79-k10-m1-b10

In [3]:
result.to_csv("ampl_result_summary.csv")